In [ ]:
import numpy as np
import gymnasium as gym

from stable_baselines3 import PPO

from security_env import SecurityEnv

path = r"C:\Users\Tuan Anh HSLU\OneDrive - Hochschule Luzern\Desktop\HSLU22\Bachelor Thesis\ML Models\models\run_default_20250421_212017\best_model\best_model.zip"

def suggest_config_for_user(user_config: np.ndarray,
                            model_path: str = path,
                            n_steps: int = 100):
    """
    1. Loads the pre-trained RL model (PPO).
    2. Resets the SecurityEnv to the user's config. 
    3. Steps through the environment using the model's policy.
    4. Returns the final proposed configuration plus any info.
    """
    # Create the environment
    env = SecurityEnv(
        alpha=0.7,  # or your chosen alpha
        beta=0.3,   # or your chosen beta
        s_min=5.0
    )

    # Load your trained PPO agent
    model = PPO.load(model_path, env=env)
    # Reset environment to the user config
    obs, info = env.reset_with_user_config(user_config)
    

    final_obs = None
    for step in range(n_steps):
        action, _states = model.predict(obs, deterministic=False)
        obs, reward, done, truncated, info = env.step(action)
        final_obs = obs  # keep the latest state
        if done or truncated:
            break

    if final_obs is None:
        raise RuntimeError("Model did not perform any step; final state is undefined.")
    
    # The 'obs' now contains the final state after n_steps, i.e. the final config + [afs, security]
    final_config = obs[:-2]  # everything except the last two (fatigue, security)
    predicted_fatigue = obs[-2]
    security_score = obs[-1]
    
    # Convert final_config to integers for compact representation
    final_config_int = np.round(final_config).astype(np.int64).tolist()
    
    # Create a compact response with just the essential information
    suggestion = {
        "config": final_config_int,  # The configuration as integers
        "fatigue": float(predicted_fatigue),
        "security": float(security_score),
        "steps": step + 1
    }
    
    # Optional: For debugging, you can still generate the detailed feature values
    if False:  # Set to True for debugging
        # Convert final_config to integers before passing to _get_feature_values
        final_config_int_arr = final_config.astype(np.int64)
        # Convert final_config from indexes back to meaningful feature values
        feature_values = env._get_feature_values(final_config_int_arr)
        
        print("\n🔎 Mapping Check: Index → Value")
        for i, feature in enumerate(env.feature_names):
            idx = int(round(final_config[i]))
            expected = env.feature_ranges[feature][idx]
            actual = feature_values[feature]
            print(f"{feature}: Index = {idx}, Mapped = {expected}, FeatureValue = {actual}, Match = {expected == actual}")
        
        suggestion["feature_values"] = feature_values
    
    return suggestion

if __name__ == "__main__":
    # Example user config: each number is the "index" for that feature 
    # in the environment's defined range
    example_user_config = np.array([
        1,  # Familarity
        1,  # Frequency of Password Changes
        1,  # Difficulty Level Password
        1,  # Effort Required Password
        1,  # Perceived Importance Password
        1,  # Password Uniqueness
        1,  # Frequency of MFA prompts
        1,  # Difficulty Level MFA
        1,  # Effort Required MFA
        1,  # Perceived Importance of MFA
        1,  # Frequency of Security Warnings
        1,  # Difficulty Level Security Warnings
        1,  # Effort Required Security Warnings
        1,  # Perceived Importance of Security Warnings
        1,  # Warnings Response Behaviour
        0,  # Hardware security key (FIDO2 token) or cryptographic device
        0,  # On-device prompt or biometric
        0,  # OTP via authenticator app
        1,  # OTP via SMS/email
        0,  # Secondary email/phone or security questions
        0   # No MFA enabled
    ])

    suggestion_output = suggest_config_for_user(example_user_config, path)
    print("\nSuggestion Output:")
    print(suggestion_output)


Feature names: ['Familarity', 'Frequency of Password Changes', 'Difficulty Level Password', 'Effort Required Password', 'Perceived Importance Password', 'Password Uniqueness', 'Frequency of MFA prompts', 'Difficulty Level MFA', 'Effort Required MFA', 'Perceived Importance of MFA', 'Frequency of Security Warnings', 'Difficulty Level Security Warnings', 'Effort Required Security Warnings', 'Perceived Importance of Security Warnings', 'Warnings Response Behaviour', 'Hardware security key (FIDO2 token) or cryptographic device', 'On-device prompt or biometric', 'OTP via authenticator app', 'OTP via SMS/email', 'Secondary email/phone or security questions', 'No MFA enabled']
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.

Suggestion Output:
{'config': [0, 2, 3, 0, 4, 2, 4, 4, 3, 3, 2, 3, 2, 1, 2, 0, 1, 0, 0, 0, 0], 'fatigue': 8.071110725402832, 'security': 76.19999694824219, 'steps': 100}

Suggestion Output:
{'config': [0, 2, 3, 0, 4, 2, 4, 4, 3, 3, 2, 3, 2, 1, 

# Testing the Configuration Suggestion Functions

This notebook demonstrates how to use the configuration suggestion functions to generate optimized security settings based on user input.

In [4]:
# Demonstrate different approaches
from IPython.display import display
import pandas as pd

# Create an example user configuration
test_user_config = np.array([
    1,  # Familarity - Intermediate
    1,  # Frequency of Password Changes - Low frequency
    1,  # Difficulty Level Password - Easy
    1,  # Effort Required Password - Low effort
    1,  # Perceived Importance Password - Somewhat important
    1,  # Password Uniqueness - Somewhat unique
    1,  # Frequency of MFA prompts - Low frequency
    1,  # Difficulty Level MFA - Easy
    1,  # Effort Required MFA - Low effort
    1,  # Perceived Importance of MFA - Somewhat important
    1,  # Frequency of Security Warnings - Low frequency
    1,  # Difficulty Level Security Warnings - Easy
    1,  # Effort Required Security Warnings - Low effort
    1,  # Perceived Importance of Security Warnings - Somewhat important
    1,  # Warnings Response Behaviour - Sometimes follow
    0,  # Hardware security key (FIDO2 token) or cryptographic device - No
    0,  # On-device prompt or biometric - No
    0,  # OTP via authenticator app - No
    1,  # OTP via SMS/email - Yes
    0,  # Secondary email/phone or security questions - No
    0   # No MFA enabled - No
])

# Create environment to get feature names
env = SecurityEnv(alpha=0.7, beta=0.3, s_min=5.0)

# Test basic suggestion function
basic_result = suggest_config_for_user(test_user_config)

# Test ensemble approach
ensemble_result = suggest_config_for_user1(test_user_config)

# Test constrained optimization
constrained_result = suggest_config_with_constraints(
    test_user_config, 
    max_allowed_fatigue=40.0,
    min_required_security=60.0
)

# Create a function to display results nicely
def display_config_comparison(user_config, result, feature_names):
    # For basic and ensemble results
    if "config" in result:
        optimized_config = result["config"]
    else:
        optimized_config = result["optimized_config"]
        
    # Create a DataFrame to display the comparison
    df = pd.DataFrame({
        "Feature": feature_names,
        "User Config": user_config,
        "Optimized Config": optimized_config
    })
    
    # Add a column showing which features were changed
    df["Changed"] = df["User Config"] != df["Optimized Config"]
    
    # Display the result metrics
    print(f"Fatigue Score: {result['fatigue']:.2f}")
    print(f"Security Score: {result['security']:.2f}")
    
    if "meets_constraints" in result:
        print(f"Meets Constraints: {result['meets_constraints']}")
    
    # Return only the rows where changes were made
    return df[df["Changed"]]

# Display results
print("\n=== Basic Suggestion Results ===")
changed_features = display_config_comparison(test_user_config, basic_result, env.feature_names)
display(changed_features)

print("\n=== Ensemble Suggestion Results ===")
changed_features = display_config_comparison(test_user_config, ensemble_result, env.feature_names)
display(changed_features)

print("\n=== Constrained Suggestion Results ===")
changed_features = display_config_comparison(test_user_config, constrained_result, env.feature_names)
display(changed_features)

Feature names: ['Familarity', 'Frequency of Password Changes', 'Difficulty Level Password', 'Effort Required Password', 'Perceived Importance Password', 'Password Uniqueness', 'Frequency of MFA prompts', 'Difficulty Level MFA', 'Effort Required MFA', 'Perceived Importance of MFA', 'Frequency of Security Warnings', 'Difficulty Level Security Warnings', 'Effort Required Security Warnings', 'Perceived Importance of Security Warnings', 'Warnings Response Behaviour', 'Hardware security key (FIDO2 token) or cryptographic device', 'On-device prompt or biometric', 'OTP via authenticator app', 'OTP via SMS/email', 'Secondary email/phone or security questions', 'No MFA enabled']
Feature names: ['Familarity', 'Frequency of Password Changes', 'Difficulty Level Password', 'Effort Required Password', 'Perceived Importance Password', 'Password Uniqueness', 'Frequency of MFA prompts', 'Difficulty Level MFA', 'Effort Required MFA', 'Perceived Importance of MFA', 'Frequency of Security Warnings', 'Diffi

,Feature,User Config,Optimized Config,Changed
0,Familarity,1,2,True
1,Frequency of Password Changes,1,0,True
2,Difficulty Level Password,1,4,True
3,Effort Required Password,1,3,True
5,Password Uniqueness,1,2,True
6,Frequency of MFA prompts,1,3,True
7,Difficulty Level MFA,1,0,True
9,Perceived Importance of MFA,1,4,True
10,Frequency of Security Warnings,1,3,True
11,Difficulty Level Security Warnings,1,3,True



=== Ensemble Suggestion Results ===
Fatigue Score: 6.89
Security Score: 85.20


,Feature,User Config,Optimized Config,Changed
1,Frequency of Password Changes,1,3,True
2,Difficulty Level Password,1,3,True
3,Effort Required Password,1,2,True
4,Perceived Importance Password,1,3,True
5,Password Uniqueness,1,2,True
6,Frequency of MFA prompts,1,4,True
7,Difficulty Level MFA,1,4,True
9,Perceived Importance of MFA,1,4,True
10,Frequency of Security Warnings,1,2,True
11,Difficulty Level Security Warnings,1,2,True



=== Constrained Suggestion Results ===
Fatigue Score: 1.11
Security Score: 34.00
Meets Constraints: False


,Feature,User Config,Optimized Config,Changed
1,Frequency of Password Changes,1,0,True
4,Perceived Importance Password,1,3,True
17,OTP via authenticator app,0,1,True
20,No MFA enabled,0,1,True
